# Propósito
O objetivo deste kernel é encontrar uma maneira de agrupar um conjunto de estudantes de acordo com as suas notas em determinadas disciplinas.

# Conteúdo
O [conjunto de dados](https://www.kaggle.com/spscientist/students-performance-in-exams) utilizado pertence ao usuário [**SPScientist**](https://www.kaggle.com/spscientist), e contém os seguintes campos:

* Gender (Gênero)
* Race/Ethnicity (Raça/Etnia)
* Parental level of education (Nível de educação dos pais)
* Lunch (Alimentação)
* Test preparation course (Curso preparatório)
* Math score (Nota em matemática)
* Reading score (Nota em leitura)
* Writing score (Nota em escrita)

# Abordagem
Como a ideia era utilizar um algoritmo de agrupamento não-supervisionado para a análise, optamos pelo k-means.

# Código
**Importando as bibliotecas necessárias**

In [ ]:
import pandas as pd
import seaborn as sns
import sklearn.metrics
import matplotlib.pyplot as plt
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import warnings

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings("ignore")

**Carregando o dataset**

In [ ]:
data = pd.read_csv('../input/StudentsPerformance.csv')

**Preparação dos dados para treinamento**

Dando uma olhada nas colunas

In [ ]:
print("Colunas: {0}\n".format(data.columns))

Renomeação das colunas para facilitar a manipulação de dados.

In [ ]:
data = data.rename(columns={'race/ethnicity' : 'ethnicity',
                            'parental level of education' : 'parentalLevelEducation',
                            'test preparation course' : 'preparationCourse',
                            'math score' : 'mathScore',
                            'reading score' : 'readingScore',
                            'writing score': 'writingScore'})

print("Colunas: {0}\n".format(data.columns))

Informações sobre o conjunto: não há dados nulos e as três colunas de notas são do tipo int64, enquanto as demais são do tipo object(string)

In [ ]:
print(data.info())

Exibindo informações gerais

In [ ]:
print("Informações sobre a tabela\n{0}".format(data.describe(include='all')))

O conjunto possui 1000 entradas.

In [ ]:
print("Formato do conjunto de dados {0}".format(data.shape))

Dando uma olhada na tabela após as alterações

In [ ]:
data.head()

**Análise exploratória de dados**

Exibição de gráficos de dispersão para alguns parâmetros do conjunto de dados

In [ ]:
sns.pairplot(data, kind="scatter", hue="gender")
plt.show()

In [ ]:
sns.pairplot(data, kind="scatter", hue="ethnicity")
plt.show()

In [ ]:
sns.pairplot(data, kind="scatter", hue="parentalLevelEducation")
plt.show()

In [ ]:
sns.pairplot(data, kind="scatter", hue="preparationCourse")
plt.show()

Exibição das distribuições de alguns parâmetros

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(16, 5), sharey=True)
sns.countplot(data.gender, ax = axs[0])
ax0 = sns.countplot(data["ethnicity"], ax = axs[1])
ax0.set_xticklabels(ax0.get_xticklabels(), rotation=90)
ax1 = sns.countplot(data["parentalLevelEducation"], ax = axs[2])
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=90)
sns.countplot(data.lunch, ax = axs[3])
sns.countplot(data["preparationCourse"], ax = axs[4])

**Aplicação do K-means** 

Separação dos parâmetros de interesse para a análise

In [ ]:
scores = data.loc[:,["mathScore","readingScore","writingScore"]]

Executando o k-means com o número de clusters variando de 1 até 10, e análise dos valores da soma dos quadrados intra-clusters (WCSS) para cada resultado

In [ ]:
wcss = []
print('Valores do WCSS:')
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, random_state = 0)
    kmeans.fit(scores)
    print(kmeans.inertia_)
    wcss.append(kmeans.inertia_)

Gráfico do joelho montado a partir dos valores de WCSS obtidos

In [ ]:
plt.plot(range(1, 11), wcss)
plt.xlabel('Número de clusters')
plt.ylabel('WCSS')

Execução do k-means com o número de clusters ideal e adição das predições ao conjunto de dados

In [ ]:
kmeans = KMeans(n_clusters = 5, random_state = 0)
previsoes = kmeans.fit_predict(scores)
data['class'] = previsoes
print(data)
plt.show()

# Apresentação dos resultados

Vizualização gráfica dos grupos gerados a partir das notas

In [ ]:
sns.pairplot(data, hue='class', diag_kind=None)

Representação das classes utilizando o [sistema de notas escolares dos Estados Unidos](https://www.mooxye.com/blog/en/american-grading-system-and-gpa/)
* **A** – De 90 a 100 pontos, corresponde à classe 2;
* **B** – De 80 a 89 pontos, corresponde à classe 0;
* **C** – De 70 a 79 pontos, corresponde à classe 1;
* **D** – De 60 a 69 pontos, corresponde à classe 3;
* **F** – Abaixo de 59 pontos, corresponde à classe 4;

In [ ]:
data.loc[data['class'] == 2, 'grade'] = "A"
data.loc[data['class'] == 0, 'grade'] = "B"
data.loc[data['class'] == 1, 'grade'] = "C"
data.loc[data['class'] == 3, 'grade'] = "D"
data.loc[data['class'] == 4, 'grade'] = "F"

sns.pairplot(data, hue='grade', diag_kind='hist')

Distribuição de cada um dos grupos

In [ ]:
fig, axs = plt.subplots()
sns.countplot(data.grade)